# <span style="color:DarkSeaGreen">Lab 3 - AgentCore</span>
- this notebook creates the following:
  - AgentCore environment to deploy the Strands Agents

In [ ]:
### START ###
### MAKE SURE YOU ARE IN THE RIGHT FOLDER WHERE THE REQUIREMENTS FILE IS LOCATED ###
### COPY FROM HERE ONLY IF RUNNING AS ONE COPY AND PASTE ###
# activate the virtual environment
source venv-agentcore/bin/activate
### COPY TO HERE ONLY IF RUNNING AS ONE COPY AND PASTE ###

### STOP ###
### MAKE SURE ABOVE VENV GETS ACTIVATED BEFORE RUNNING THE REST ###
# install the required packages - may need to specify the path here if not in the correct folder in terminal window
pip install -r requirements_lab3.txt
# pip install -r Documents/github/labs-sagemaker/jumpstart/requirements_lab3.txt
# verify the installation
pip list

### RESTART VSCODE TO PICKUP THE NEW VENV ###

In [ ]:
### STOP ###
### This command is for activating an environment that already exists, its for use in a terminal window if you need it ###
source venv-agentcore/bin/activate
pip list

# use pip freeze if you prefer for friendly format
### ALSO MAKE SURE YOU SELECT IT AS YOUR KERNEL FOR THIS JUPYTER NOTEBOOK ###

# <span style="color:DarkSeaGreen">AgentCore MCP Server</span>
- If you are using an environment that supports use of MCP Servers, you can install the AgentCore MCP Server to provide natural language interfaces for AgentCore development
- For example, this notebook was developed using the help of the AgentCore MCP Server by installing it into VSCode Amazon Q Chatbot
- https://aws.amazon.com/blogs/machine-learning/accelerate-development-with-the-amazon-bedrock-agentcore-mcpserver/

In [ ]:
Name: bedrock-agentcore-mcp-server
Transport: stdio
Command: uvx
Arguments: awslabs.amazon-bedrock-agentcore-mcp-server@latest

# Lab 3 Starts Here!

# <span style="color:DarkSeaGreen">AgentCore</span>

- We will deploy the 4 Strands agents and the Strands orchestration agent into AgentCore
- We will use the AgentCore Python SDK to do this via boto3
- AgentCore hosts them as containerized applications
  - Strands agents can be deployed to AgentCore as-is without converting them to Lambda functions. AgentCore uses a different deployment model:
    - Container-based deployment : Your Python scripts are packaged into containers and run on AgentCore Runtime
    - No Lambda conversion needed : The agents run as containerized applications, not serverless functions
    - Built-in orchestration : You can deploy multiple agents and route between them

